In [1]:
# MESA - related imports
from collections import defaultdict
from mesa import Agent
from mesa import Model
from mesa.batchrunner import BatchRunner, FixedBatchRunner, ParameterSampler
from mesa.datacollection import DataCollector
from mesa.time import StagedActivation
from mesa.space import MultiGrid

# Other imports
import copy
import numpy as np
import pandas as pd
import random
from statistics import mean

# Used for sampling inputs in BatchRunnerRandomParam
from SALib.sample import saltelli

## Agents
Cell, Jurisdiction, Household

In [2]:
# Cell types
WATER = 0
LAND  = 1

# 1 step is 3 months worth of time
STEP_SIZE = 3

In [3]:
'''
Cell class used for populating the environment grid. Cells don't perform 
any actions of their own, they simply keep track of the environment grid state.
'''

class Cell(Agent):

    def __init__(self, pos, celltype, model, init_type=0):

        super().__init__(pos, model) # uses the cell's (x,y) position as the unique_id
        self.x, self.y = pos
        self.jurisdiction = None
        self.celltype  = celltype
        self.flooded   = False


    def step(self):
        pass


    '''
    Staged Activation
    '''
    def stage0(self):
        pass

    def stage1(self):
        pass

    def stage2(self):
        pass


In [4]:
'''
Jurisdiction class
'''

class Jurisdiction(Agent):

    def __init__(self, unique_id, model, policy_mechanism):
        super().__init__(unique_id, model)

        # MEAN or MAJORITY
        self.policy_mechanism = policy_mechanism

        # initiate random policy
        self.policy = self.model.random.choice(self.model.policies)
        
        # set up different voting mechanism
        if self.policy_mechanism == 'MEAN':
            self.votes = []
            
        elif self.policy_mechanism == 'MAJORITY':
            self.votes = {0.0  : 0,
                          0.25 : 0,
                          0.5  : 0,
                          0.75 : 0,
                          1.0  : 0}

        self.avg_utility = None

        # determine initial tax
        self.max_tax = self.model.max_tax #0.05
        self.tax = None
        self.update_tax()

        # flooding trackers
        self.flooded = False
        
        # determine initial protection level
        self.flood_damage = None
        self.update_protection()

        # determine the center of the household cluster
        while True:
            # select random (x,y) coordinate from the jurisdiction's cells
            self.center_x = self.model.random.choice(self.model.jurisdictions[self.unique_id]['x'])
            self.center_y = self.model.random.choice(self.model.jurisdictions[self.unique_id]['y'])
            # if the chosen coordinate is a land cell, pick it
            if self.model.data[self.center_x][self.center_y] == 1:
                break



    '''
    Functions for updating policy, with corresponding tax and protection level
    '''
    def update_policy_mean(self):
        # if there are votes, set the new policy to the means of the votes
        if self.votes != []:
            self.policy = mean(self.votes)
        # if there are no votes the jurisdiction must be empty. Reset policy randomly
        else:
            self.policy = self.model.random.uniform(0,1)


    def update_policy_majority(self):
        # if there are votes, pick the policy with the most votes
        if sum(self.votes.values()) != 0:
            max_votes = max(self.votes.values())
            max_policies = [p for p, v in self.votes.items() if v == max_votes]

            # if there is a tie for most popular policy, pick the one closer to the current policy
            if len(max_policies)>1:
                 self.policy = min(max_policies, key=lambda x:abs(x-self.policy))
            else:
                self.policy = max_policies[0]
        # if there are no votes the jurisdiction must be empty. Reset policy randomly
        else:
            self.policy = self.model.random.choice(self.model.policies)


    def update_tax(self):
        # in [0 ... self.max_tax]
        self.tax = self.max_tax * self.policy


    def update_protection(self):
        # reduce the base flood damage depending on the policy
        self.flood_damage = self.model.base_flood_damage * (1-self.policy)


    '''
    Generate a new location for a household based on the cluster center
    '''
    def new_location(self):
        x,y = self.model.generate_point(self.center_x, self.center_y, 100, 100)

        while len(self.model.grid.get_cell_list_contents((x, y))) == 1 \
                  or x not in self.model.jurisdictions[self.unique_id]['x'] \
                  or y not in self.model.jurisdictions[self.unique_id]['y']:
            x,y = self.model.generate_point(self.center_x, self.center_y, 100, 100)
                
        return (x,y)


    def flood(self):
        self.flooded = True
        

    def unflood(self):
        self.flooded = False


    '''
    Step function : not used
    '''
    def step(self):
        pass


    '''
    Staged Activation 
    '''
    def stage0(self):
        # clear the votes from the previous cycle
        if self.policy_mechanism == "MEAN":
            self.votes = []
        elif self.policy_mechanism == "MAJORITY":
            self.votes = {0    : 0,
                          0.25 : 0,
                          0.5  : 0,
                          0.75 : 0,
                          1    : 0}
        else:
            print("There was an error with the policy_mechanism string.")
        

    def stage1(self):
        # determine average utility in the jurisdiction
        self.avg_utility = self.model.datacollector.model_vars['avg_U_JUR'][int(self.model.schedule.time)-1][self.unique_id]

            
        # update state based on latest voting round
        if self.policy_mechanism == 'MEAN':
            self.update_policy_mean()
        elif self.policy_mechanism == 'MAJORITY':
            self.update_policy_majority()
        else:
            print("There is an error with the policy mechanism")
        self.update_tax()
        self.update_protection()

    def stage2(self):
        pass


In [5]:
'''
Household agent. Represents both the physical house and the people living in it
'''

class Household(Agent):

    def __init__(self, unique_id, pos, model, cell, jurisdiction, inc_quintile, inc_annual, inc_monthly, savings_months, total_savings_dollars, monthly_savings_dollars, vote):
        super().__init__(unique_id, model)
        self.x, self.y = pos
        self.cell = cell
        self.jurisdiction = jurisdiction
        
        # money-related state
        self.inc_quintile = inc_quintile
        self.inc_annual = inc_annual
        self.inc_monthly = inc_monthly

        self.savings_months = savings_months
        self.total_savings_dollars = total_savings_dollars
        self.monthly_savings_dollars = monthly_savings_dollars

        self.income_leftover = 1

        # desired level of public flood protection
        self.vote = vote

        # how many times has my jurisdiction disappointed me?
        self.times_moved = 0
        self.discontent = 0
        self.max_discontent = self.model.random.randint(6, 10)

        # starting utility, recalculated at every step
        self.utility = self.calculate_utility(self.jurisdiction)

        # in this time step, not used in calculations but tracked in the data
        self.flood_damage_received = 0



    '''
    Cast agent vote in its home jurisdiction
    '''
    def cast_vote(self):
        if self.jurisdiction.policy_mechanism == "MEAN":
            self.jurisdiction.votes.append(self.vote)
        elif self.jurisdiction.policy_mechanism == "MAJORITY":
            # pick the voting option that is closest to my preference
            option = min(self.model.policies, key=lambda x:abs(x-self.vote))
            # cast my vote for that option
            self.jurisdiction.votes[option] += 1



    '''
    Utility function for a household in a particular jurisdiction.
    U = x^a * y^b
    x : level of safety (self.jurisdiction.policy)
    y : income minus flood tax
    a : percentage of income spent on flood tax
    b : percentage of income not spent on flood tax
    '''
    def calculate_utility(self, jur):
        x = jur.policy
        y = self.inc_monthly * (1-jur.tax)

        a = jur.tax
        b = self.income_leftover

        U = pow(x, a) * pow(y, b)
        return U



    '''
    Decide whether to stay in the current jurisdiction or move to a new one.
    Households calculate their satisfaction with a jurisdiction based on the utility function.
    '''
    def moving_decision(self):

        # Utility in my current jurisdiction (not a necessary calculation)
        self.utility = self.calculate_utility(self.jurisdiction)


        # if my utility is lower than the jurisdiction's average
        if self.utility < self.jurisdiction.avg_utility:
            self.discontent += 1
        else:
            self.discontent -= 1

        # if I have been discontent with my jurisdiction for too long
        if self.discontent >= self.max_discontent:
            # find a new jurisdiction with a higher utility
            jurisdictions = [self.model.schedule._agents[i] for i in range(self.model.num_jur)]
            utilities = []
            for jur in jurisdictions:
                utilities.append(self.calculate_utility(jur))

            return max(enumerate(utilities), key=lambda x:x[1])[0] # id of the jurisdiction with highest utility level

        return self.jurisdiction.unique_id



    '''
    Migrates household from its current cell to a new cell in a specified jurisdiction (by id)
    '''
    def move(self, jurisdiction: int):
        if jurisdiction == self.jurisdiction.unique_id:
            return

        # the HH can only move if their savings are enough to cover the moving cost
        if self.savings_months < self.model.moving_cost:
            return

        # pick a random cell within the new jurisdiction that is available and not on water
        new_jur = self.model.schedule._agents[jurisdiction]
        new_x, new_y = new_jur.new_location()

        # move to my chosen cell, update state
        self.model.grid.move_agent(self, (new_x, new_y))
        self.cell = self.model.schedule._agents[(new_x, new_y)]
        self.jurisdiction = new_jur
        self.x = new_x
        self.y = new_y
        self.discontent = 0
        self.times_moved += 1
        
        # update savings variables
        self.savings_months -= self.model.moving_cost
        self.total_savings_dollars -= self.model.moving_cost * self.inc_monthly
        


    '''
    Administer flood damage to the household in the form of savings loss
    '''
    def flood_damage(self):
        self.flood_damage_received += self.jurisdiction.flood_damage * self.inc_monthly * STEP_SIZE
        self.total_savings_dollars -= self.flood_damage_received

        
    '''
    Calculates the proportion of income (per step) that is left at a household's disposal
    after paying flood tax and (if needed) covering some of its debt.
    
    Note: living espenses + savings + what's left after paying debt count as income_leftover
    
    Note: 1 step = 3 months but since we are calculating a PROPORTION of income, it is 
    equivalent to working with the quantities for a single month.
    '''
    def calculate_income_leftover(self):
        # flood tax
        tax_expense = self.inc_monthly * self.jurisdiction.tax
        
        # if in debt (negative savings)
        if self.total_savings_dollars < 0:
            
            # fixed expenses: 80% of income for living expenses + monthly savings
            base_expense = (self.inc_monthly * 0.8) + self.monthly_savings_dollars
            
            # calculate the amount of money at my disposal to repay my debts
            debt_money = self.inc_monthly - base_expense - tax_expense
            
            # if it isn't enough to cover the debt in one go, the debt is only partially repaid. 
            debt_money_leftover = self.total_savings_dollars + debt_money

            if debt_money_leftover <= 0:
                self.total_savings_dollars += debt_money
                self.income_leftover = base_expense / self.inc_monthly
            
            # if it is enough to cover the debt in one go, the debt is removed and there is some money left over
            elif debt_money_leftover > 0:
                self.total_savings_dollars = 0
                self.income_leftover = (base_expense + debt_money_leftover) / self.inc_monthly
                
        # if not in debt, income leftover is everything except tax
        else:
            self.income_leftover = (self.inc_monthly - tax_expense) / self.inc_monthly
        
        
    '''
    Update savings at each step (fixed amount added each step)
    1 step = 3 months (STEP_SIZE)
    '''
    def add_savings(self):
        self.total_savings_dollars += STEP_SIZE * self.monthly_savings_dollars
        self.savings_months = self.total_savings_dollars / self.inc_monthly

        
        
    '''
    Step method : not used
    '''
    def step(self):
        pass

    '''
    Staged Activation
    '''
    def stage0(self):
        # reset income leftover at the start of the step
        self.income_leftover = 1
        self.flood_damage_received = 0
        self.cast_vote()

    def stage1(self):
        pass

    def stage2(self):
        if self.jurisdiction.flooded:
            self.flood_damage()
        
        self.calculate_income_leftover()
        self.add_savings()
        
        self.utility = self.calculate_utility(self.jurisdiction)

        jurisdiction = self.moving_decision()
        if jurisdiction != self.jurisdiction.unique_id:
            self.move(jurisdiction)


In [6]:
'''
A scheduler which activates each type of agent once per step, in random
order, with the order reshuffled every step.

This is equivalent to the NetLogo 'ask type...' and is generally the
default behavior for an ABM.

Order of calling: Jurisdiction, Cell, Household
'''

class StagedActivationByType(StagedActivation):

    def __init__(self, model, stage_list, shuffle=False, shuffle_between_stages=False):
        super().__init__(model, stage_list, shuffle, shuffle_between_stages)
        self.agents_by_type = defaultdict(dict)



    def add(self, agent):
        """
        Add an Agent object to the schedule
        Args:
            agent: An Agent to be added to the schedule.
        """

        self._agents[agent.unique_id] = agent
        agent_class = type(agent)
        self.agents_by_type[agent_class][agent.unique_id] = agent



    def remove(self, agent):
        """
        Remove all instances of a given agent from the schedule.
        """

        del self._agents[agent.unique_id]

        agent_type = type(agent)
        del self.agents_by_type[agent_type][agent.unique_id]



    def step(self):
        for stage in self.stage_list:    

            for agent_type in self.agents_by_type:
                agent_keys = list(self.agents_by_type[agent_type].keys())
                if self.shuffle:
                    self.model.random.shuffle(agent_keys)
                for agent_key in agent_keys:
                    getattr(self._agents[agent_key], stage)()
                    
            if self.shuffle_between_stages:
                self.model.random.shuffle(agent_keys)
            self.time += self.stage_time
        self.steps += 1


## Model-level data collection

In [7]:
'''
RQ1: Household satisfaction with the policy
The average distance for all households between their desired policy and actual policy
'''
def HH_policy_satisfaction(model):
    households = [a for a in model.schedule.agents if type(a) == Household]

    # if the distance is positive, households want more protection than they are getting
    # negative values are giving me an error in sobol, so I will switch to using absolute 
    # distance. It doesn't give us the direction but it's some measure of distance
    distances = [abs(hh.vote - hh.jurisdiction.policy) for hh in households]
    
    return mean(distances)



'''
RQ2: Household utility
The average utility for all households
'''
def avg_U(model):
    households = [a for a in model.schedule.agents if type(a) == Household]
    utilities = [hh.utility for hh in households]
    return mean(utilities)



'''
RQ3: Level of public protection
The average level of public protection for the whole model
'''
def avg_flood_protection(model):
    flps = []
    for i in range(model.num_jur):
        flps.append(model.schedule._agents[i].policy)
    
    return mean(flps)



'''
Average utility in each jurisdiction.
return : [avg_U0, avg_U1, avg_U2...]
'''
def avg_U_JUR(model):
    avg_utilities = []
    for i in range(model.num_jur):
        households = [a for a in model.schedule.agents if type(a) == Household and a.jurisdiction.unique_id == i]
        if households == []:
            avg_utilities.append(1)
        else:
            avg_utilities.append(mean([hh.utility for hh in households]))

    return avg_utilities

## Model

In [8]:
'''
Model class
'''
class FloodModel(Model):

    def __init__(self, height, width, water_p, land_p, policy_mechanism, model_seed, moving_cost, max_tax, base_flood_damage):
        
        # Set the random seed
        self.reset_randomizer(model_seed)

        # Initializing model parameters
        self.height  = height
        self.width   = width 

        self.water_p = water_p
        self.land_p  = land_p

        self.num_hh  = 1500
        self.moving_cost = moving_cost
        self.max_tax = max_tax
        
        # Flood damage as a proportion of quarterly income
        self.base_flood_damage = base_flood_damage #1.5
        self.flooded = False
        
        # translating saltelli sample numbers into scenario / policy mechanism
        if policy_mechanism==0:
            self.policy_mechanism = "MEAN"
        elif policy_mechanism==1:
            self.policy_mechanism = "MAJORITY"
            
        
        # possible flood protection levels
        self.policies = [0.0, 0.25, 0.5, 0.75, 1.0]

        self.stage_list = ["stage0", "stage1", "stage2"]
        self.schedule = StagedActivationByType(self, self.stage_list)

        self.grid = MultiGrid(self.width, self.height, torus=True)

        # Preparing the map's starting state
        # Creating different natural zones
        data_water = [[WATER for i in range(self.width)] for i in range(int(self.water_p*self.height))]
        data_land = [[LAND for i in range(self.width)] for i in range(int(self.land_p*self.height))]

        self.data = data_water + data_land

        # Fading between zones
        self.random_fade(self.data, self.water_p, WATER)


        # Contents of each jurisdiction:
        self.jurisdictions = {
                            0 : {'x': list(range(int(self.width))),
                                 'y': list(range(int(self.height/4)))} ,

                            1 : {'x': list(range(int(self.width))),
                                 'y': list(range(int(self.height/4), int(self.height/2)))},

                            2 : {'x': list(range(int(self.width))),
                                 'y': list(range(int(self.height/2), int(3*self.height/4)))},

                            3 : {'x': list(range(int(self.width))),
                                 'y': list(range(int(3*self.height/4), self.height))}
                            }

        # Creating Jurisdiction agents
        self.num_jur = len(self.jurisdictions)
        for i in range(self.num_jur):
            a = Jurisdiction(i,self, self.policy_mechanism)
            self.schedule.add(a)


        # Populating the grid with Cell agents
        for (contents, x, y) in self.grid.coord_iter():
            # pass in self.data to assign cell type
            cell = Cell((x,y), self.data[x][y], self)

            # Assign cells to jurisdictions
            for i in range(self.num_jur):
                if x in self.jurisdictions[i]['x'] and y in self.jurisdictions[i]['y']:
                    cell.jurisdiction = self.schedule._agents[i]

            self.grid.place_agent(cell, (x,y))
            self.schedule.add(cell)


        # Populating the grid with Household agents
        # Centers of Household clusters, one per jurisdiction
        centers = []
        for i in range(self.num_jur):
            jur = self.schedule._agents[i]
            centers.append((jur.center_x, jur.center_y))

        # Determine Household positions in the grid based on center points
        points = self.clusters(centers)
        # Shuffle household positions to ensure random assignment to households
        #random.shuffle(points)
        self.random.shuffle(points)

        
        # Create a dataframe containing the household data
        population = pd.read_csv(r'data/Household_Population.csv')
        
        
        # Place Household agents in the grid
        i = 0
        uid = self.num_jur
        for (x,y) in points:

            # Assign household to the same jurisdiction as its Cell
            cell = self.grid.get_cell_list_contents((x,y))[0]
            jur = cell.jurisdiction 

            hh = Household(uid, 
                (x,y), 
                self, 
                cell, 
                jur,
                population['inc_quintile'][i],
                population['inc_annual'][i],
                population['inc_monthly'][i],
                population['savings_months'][i],
                population['total_savings_dollars'][i],
                population['monthly_savings_dollars'][i],
                population['vote'][i])

            self.grid.place_agent(hh, (x,y))
            self.schedule.add(hh)
            uid += 1
            i += 1
    
        # Data collection
        self.datacollector = DataCollector(
            model_reporters = {"HH_policy_satisfaction": HH_policy_satisfaction,
                               "avg_U"                 : avg_U,
                               "avg_U_JUR"             : avg_U_JUR,
                               "avg_flood_protection"  : avg_flood_protection})

        
        self.running = True

    '''
    Helper functions for creating household clusters
    '''
    def generate_point(self, mean_x, mean_y, deviation_x, deviation_y):
        x = int(self.random.gauss(mean_x, deviation_x))
        y = int(self.random.gauss(mean_y, deviation_y))
        while x < 0 or x >= self.width:
            x = int(self.random.gauss(mean_x, deviation_x))
        while y < 0 or y >= self.width:
            y = int(self.random.gauss(mean_y, deviation_y))
        if self.data[x][y] == WATER:
            x, y = self.generate_point(mean_x, mean_y, deviation_x, deviation_y)
        return x, y


    def clusters(self, centers):
        cluster_deviation_x = 100
        cluster_deviation_y = 100
        point_deviation_x = 10
        point_deviation_y = 10

        number_of_clusters = self.num_jur
        points_per_cluster = int(self.num_hh/number_of_clusters)
        
        points = []
        
        for center_x, center_y in centers:
            for i in range(points_per_cluster):
                x,y = self.generate_point(center_x, center_y, point_deviation_x, point_deviation_y)
                # prevent duplicate household locations
                while (x,y) in points:
                    x,y = self.generate_point(center_x, center_y, point_deviation_x, point_deviation_y)
                    
                points.append((x,y))

        return points


    '''
    Functions to mark land cells as flooded, which gets rendered as blue in server.py
    NOT USED: this notebook does not start a server, there is no visualization
    '''
    def flood_cells(self):
        contents = [cell[0] for cell,x,y in self.grid.coord_iter() if cell[0].celltype == LAND]
        for cell in contents:
            cell.flooded = True



    def unflood_cells(self):
        contents = [cell[0] for cell,x,y in self.grid.coord_iter() if cell[0].celltype == LAND]
        for cell in contents:
            cell.flooded = False


    '''
    Helper function to fade between zones. Creates a "natural" seeming transition
    between water and land zones. NOT USED : no visualization in this notebook
    '''
    def random_fade(self, data, loc, celltype):
        line = int(self.height * loc)
        for i in range(line, line + int(self.height/6)):  # fading zone is 1/6 the map height
            for j in range(self.width):
                if self.random.random() > (i-line)/20:         # makes fade smoother
                    data[i][j] = celltype



    '''
    Step function 
    '''
    def step(self):
        
        # collect data
        self.datacollector.collect(self)
        
        # centrally flood at fixed steps
        if int(self.schedule.time) == 99:
            for i in range(self.num_jur):
                jurisdiction = self.schedule._agents[i]
                jurisdiction.flood()
            
        
        # centrally unflood at fixed steps
        if int(self.schedule.time) == 100:
            for i in range(self.num_jur):
                jurisdiction = self.schedule._agents[i]
                jurisdiction.unflood()


        # step all agents
        self.schedule.step()
        


## BatchRunner that samples parameters randomly

In [9]:
'''
Saltelli Sampler that uses SALib's saltelli sample method to generate inputs for the model.
To be used with Sobol' sensitivity analysis
'''
class SaltelliSampler:
    def __init__(self, parameter_lists, N, random_state=None):
        self.param_names, self.param_lists = zip(
            *(copy.deepcopy(parameter_lists)).items()
        )

        self.N = N
        if random_state is None:
            self.random_state = random.Random()
        elif isinstance(random_state, int):
            self.random_state = random.Random(random_state)
        else:
            self.random_state = random_state
        self.count = 0
        
        names = list(self.param_names)
        # calculate upper and lower bounds for each parameter
        bounds = [[min(vals), max(vals)] for vals in self.param_lists]
        
        self.problem = {
            'num_vars': int(len(self.param_names)),      # number of input parameters
            'names'   : names,                           # the names of the parameters
            'bounds'  : bounds                           # upper and lower bounds
        }
        
        # Saltelli sample
        self.param_values = saltelli.sample(self.problem, N) # produces 8*N samples
        
        # Edit the "scenario" portion of the sample to be only MEAN or MAJ
        # Scenario has to be the 4th (last) parameter to be passed
        for i in range(self.param_values.shape[0]):
            if self.param_values[i][-1] > 0.5:
                self.param_values[i][-1] = 1
            elif self.param_values[i][-1] <= 0.5:
                self.param_values[i][-1] = 0
        
        
        self.runs = int(self.param_values.shape[0])

    def __iter__(self):
        return self

    def __next__(self):
        self.count += 1
        if self.count <= (self.runs):
            return dict(
                zip(
                    self.param_names,
                    list(self.param_values[self.count-1]), # draw parameter values from the saltelli sample
                )
            )
        raise StopIteration()

In [10]:
'''
Same as the MESA BatchRunner but instead of ParameterProduct uses
SaltelliSampler for random input parameters compatible with Sobol' sensitivity analysis.
'''
class BatchRunnerRandomParam(FixedBatchRunner):

    def __init__(
        self,
        model_cls,
        variable_parameters=None,
        fixed_parameters=None,
        iterations=1,
        N=2**8,  # has to be power or 2. Passed to saltelli sampler
        max_steps=300,
        model_reporters=None,
        agent_reporters=None,
        display_progress=True,
    ):

        if variable_parameters is None:
            super().__init__(
                model_cls,
                variable_parameters,
                fixed_parameters,
                iterations,
                max_steps,
                model_reporters,
                agent_reporters,
                display_progress,
            )
        else:
            super().__init__(
                model_cls,
                SaltelliSampler(variable_parameters, N),
                fixed_parameters,
                iterations,
                max_steps,
                model_reporters,
                agent_reporters,
                display_progress,
            )


In [11]:
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

In [12]:
# Note: This Batch Runner uses saltelli input sampling which it inherits from BatchRunnerRandomParam
class BatchRunnerMP(BatchRunnerRandomParam):
    """Child class of BatchRunner, extended with multiprocessing support."""

    def __init__(self, model_cls, nr_processes=None, **kwargs):
        """Create a new BatchRunnerMP for a given model with the given
        parameters.
        model_cls: The class of model to batch-run.
        nr_processes: int
                      the number of separate processes the BatchRunner
                      should start, all running in parallel.
        kwargs: the kwargs required for the parent BatchRunner class
        """
        if nr_processes is None:
            # identify the number of processors available on users machine
            available_processors = cpu_count()
            self.processes = available_processors
            print("BatchRunner MP will use {} processors.".format(self.processes))
        else:
            self.processes = nr_processes

        super().__init__(model_cls, **kwargs)
        self.pool = Pool(self.processes)

    def _make_model_args_mp(self):
        """Prepare all combinations of parameter values for `run_all`
        Due to multiprocessing requirements of @StaticMethod takes different input, hence the similar function
        Returns:
            List of list with the form:
            [[model_object, dictionary_of_kwargs, max_steps, iterations]]
        """
        total_iterations = self.iterations
        all_kwargs = []

        count = len(self.parameters_list)
        if count:
            for params in self.parameters_list:
                kwargs = params.copy()
                kwargs.update(self.fixed_parameters)
                # run each iterations specific number of times
                for iter in range(self.iterations):
                    kwargs_repeated = kwargs.copy()
                    all_kwargs.append(
                        [self.model_cls, kwargs_repeated, self.max_steps, iter]
                    )

        elif len(self.fixed_parameters):
            count = 1
            kwargs = self.fixed_parameters.copy()
            all_kwargs.append(kwargs)

        total_iterations *= count

        return all_kwargs, total_iterations

    @staticmethod
    def _run_wrappermp(iter_args):
        """
        Based on requirement of Python multiprocessing requires @staticmethod decorator;
        this is primarily to ensure functionality on Windows OS and does not impact MAC or Linux distros
        :param iter_args: List of arguments for model run
            iter_args[0] = model object
            iter_args[1] = key word arguments needed for model object
            iter_args[2] = maximum number of steps for model
            iter_args[3] = number of time to run model for stochastic/random variation with same parameters
        :return:
            tuple of param values which serves as a unique key for model results
            model object
        """

        model_i = iter_args[0]
        kwargs = iter_args[1]
        max_steps = iter_args[2]
        iteration = iter_args[3]

        # instantiate version of model with correct parameters
        model = model_i(**kwargs)
        while model.running and model.schedule.steps < max_steps:
            model.step()

        # add iteration number to dictionary to make unique_key
        kwargs["iteration"] = iteration

        # convert kwargs dict to tuple to  make consistent
        param_values = tuple(kwargs.values())

        return param_values, model

    def _result_prep_mp(self, results):
        """
        Helper Function
        :param results: Takes results dictionary from Processpool and single processor debug run and fixes format to
        make compatible with BatchRunner Output
        :updates model_vars and agents_vars so consistent across all batchrunner
        """
        # Take results and convert to dictionary so dataframe can be called
        for model_key, model in results.items():
            if self.model_reporters:
                self.model_vars[model_key] = self.collect_model_vars(model)
            if self.agent_reporters:
                agent_vars = self.collect_agent_vars(model)
                for agent_id, reports in agent_vars.items():
                    agent_key = model_key + (agent_id,)
                    self.agent_vars[agent_key] = reports
            if hasattr(model, "datacollector"):
                if model.datacollector.model_reporters is not None:
                    self.datacollector_model_reporters[
                        model_key
                    ] = model.datacollector.get_model_vars_dataframe()
                if model.datacollector.agent_reporters is not None:
                    self.datacollector_agent_reporters[
                        model_key
                    ] = model.datacollector.get_agent_vars_dataframe()

        # Make results consistent
        if len(self.datacollector_model_reporters.keys()) == 0:
            self.datacollector_model_reporters = None
        if len(self.datacollector_agent_reporters.keys()) == 0:
            self.datacollector_agent_reporters = None

    def run_all(self):
        """
        Run the model at all parameter combinations and store results,
        overrides run_all from BatchRunner.
        """

        run_iter_args, total_iterations = self._make_model_args_mp()
        # register the process pool and init a queue
        # store results in ordered dictionary
        results = {}

        if self.processes > 1:
            with tqdm(total_iterations, disable=not self.display_progress) as pbar:
                for params, model in self.pool.imap_unordered(
                    self._run_wrappermp, run_iter_args
                ):
                    results[params] = model
                    pbar.update()

                self._result_prep_mp(results)
        # For debugging model due to difficulty of getting errors during multiprocessing
        else:
            for run in run_iter_args:
                params, model_data = self._run_wrappermp(run)
                results[params] = model_data

            self._result_prep_mp(results)

        # Close multi-processing
        self.pool.close()

        return (
            getattr(self, "model_vars", None),
            getattr(self, "agent_vars", None),
            getattr(self, "datacollector_model_reporters", None),
            getattr(self, "datacollector_agent_reporters", None),
        )

In [13]:
# Note: moving cost 2/3 = 2 months worth of salary

'''
This is the N passed into the Saltelli sampler. It will produce 8*N samples, 
so the simulation will run 8*N times. N has to be a power of 2.
Change to N=2 if you're just doing a test run. Otherwise it takes too long.
'''
N = 2 ** 7

# fixed parameters
fixed_params = {"height"  : 100,
                "width"   : 100,
                "water_p" : 0.3,
                "land_p"  : 0.7,
                "model_seed" : 1011} # has to be fixed


# variable parameters: upper and lower bounds
variable_params = {"moving_cost"      : [0, 1],
                   "max_tax"          : [0, 0.5],
                   "base_flood_damage": [0, 3],
                   "policy_mechanism" : [0, 1]}


batch_run = BatchRunnerMP(model_cls=FloodModel,
                          nr_processes=6,
                          variable_parameters=variable_params,
                          fixed_parameters=fixed_params,
                          iterations=1,
                          N=N,
                          max_steps=300)


batch_run.run_all()

20it [00:54,  2.71s/it]


(None,
 None,
 OrderedDict([((0.2197265625,
                0.04833984375,
                1.5556640625,
                1.0,
                100,
                100,
                0.3,
                0.7,
                1011,
                0),
                    HH_policy_satisfaction        avg_U  \
               0                  0.515667  5509.420776   
               1                  0.265500  5379.747224   
               2                  0.265500  5379.747224   
               3                  0.265500  5379.747224   
               4                  0.265500  5379.747224   
               ..                      ...          ...   
               295                0.304000  5644.204599   
               296                0.304000  5644.204599   
               297                0.304000  5644.204599   
               298                0.304000  5644.204599   
               299                0.304000  5644.204599   
               
                        

In [14]:
data_collector_agents = batch_run.get_collector_model()